# 단어, 단어 뜻, 품사, 예문, 예문 뜻 추출
        

In [ ]:
#  최종 코드
import pdfplumber
import os
import re
import json
from tqdm import tqdm

if not os.path.exists("./static"):
    os.mkdir("./static")
    
import fitz  # PyMuPDF

# 디렉토리 설정
PDF_DIR = "/Users/mane/Documents/프로젝트/eng-word/data/pdf" # pdf 넣을 폴더
OUT_DIR = "./json_words/data" # 변환된 json들이 들어갈 폴더

def extract_lesson_titles(page): # 페이지에서 "Lesson" 제목을 추출하는 함수
    text = page.extract_text()  # 페이지의 텍스트를 추출
    lesson_titles = re.findall(r'Lesson \d+\. (.+?)\s*\n', text)  # "Lesson" 번호와 제목을 정규식으로 찾음
    return lesson_titles

def extract_data_from_table(row): # 테이블의 행에서 예문과 예문 뜻을 분리하는 함수
    example_sentence, example_meaning = "", ""
    if not isinstance(row[6], str):  # Check if row[6] is a string
        return {}  # Return an empty dictionary if not a string

    if re.search(r'[A-Za-z]+.*?[.!?]\s+[가-힣]', row[6]):  # 영어와 한글이 섞인 패턴을 찾음
        parts = re.split(r'(?<=[.!?])\s+(?=[가-힣])', row[6], 1)  # 영어 문장과 한글 뜻을 분리
    elif re.search(r'[A-Za-z]+\s+[가-힣]', row[6]):
        parts = re.split(r'(?<=[A-Za-z])\s+(?=[가-힣])', row[6], 1)  # 영어 단어와 한글 뜻을 분리
    elif "\n" in row[6]:
        parts = row[6].split("\n", 1)  # 개행으로 구분된 경우 분리
    else:
        parts = [row[6].strip(), ""]  # 기본적으로 공백 제거 후 할당

    example_sentence = remove_newlines(parts[0].strip())  # 예문에서 개행 제거
    mixed = extract_sentences_with_korean(example_sentence)  # 예문에서 한국어 문장을 추출
    example_meaning = remove_newlines(parts[1].strip()) if len(parts) > 1 else ""  # 예문 뜻에서 개행 제거
    
    # 예문과 예문 뜻에서 한국어 문장이 섞여 있는 경우 처리
    if example_meaning.startswith('씨'):
        concat = example_sentence + example_meaning
        example_meaning = extract_sentences_with_korean(concat)[0]
        example_sentence = concat.replace(example_meaning, '')
    
    if len(mixed) > 0: 
        if mixed[0] in example_sentence:
            example_sentence = example_sentence.replace(mixed[0], '')
            example_meaning = mixed[0] + ' ' + example_meaning

    # 품사와 단어 뜻을 정규식으로 추출
    pos_match = re.search(r'(\w+)\)\s*', row[3])
    if pos_match:
        part_of_speech = pos_match.group(1)  # 품사 추출
        meaning_text = re.sub(r'\s*\(\w+\)\s*', '', row[3]).strip()  # 뜻에서 품사 제거
    else:
        part_of_speech = ""
        meaning_text = row[3].strip()

    # 반환할 단어 정보 사전 생성
    return {
        "단어": remove_newlines(row[0].strip()),
        "단어 뜻": meaning_text.replace(part_of_speech + ')', '').strip(),
        "품사": part_of_speech,
        "예문": example_sentence.strip(),
        "예문 뜻": example_meaning.strip(),
    }

def remove_newlines(text):  # 문자열에서 개행 문자를 제거하는 함수
    return text.replace("\n", " ").replace("\r", "")

def extract_sentences_with_korean(text):  # 영문장에서 한국어가 존재시 재 추출하는 함수
    sentences = re.split(r'(?<=[.?!])\s+', text)
    korean_sentences = [sentence for sentence in sentences if re.search(r'[가-힣]', sentence)]
    return korean_sentences

def safe_filename(filename):  # 파일 이름에서 안전하지 않은 문자를 제거하는 함수
    return re.sub(r'[\/\:*?"<>|]', "", filename)

def pdf_to_json(pdf_path, output_directory):  # beyond.pdf를 제외한 pdf용 : pdfplumber를 사용하여 PDF를 JSON으로 변환하는 함수
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages):
            lesson_titles = extract_lesson_titles(page)  # "Lesson" 제목 추출
            title_text = " - ".join(lesson_titles)  # 제목을 파일명으로 변환
            safe_title_text = safe_filename(title_text)  # 안전한 파일명 변환
            data = []
            tables = page.extract_tables()  # 페이지에서 테이블 추출
            for table in tables:
                for row in table:
                    if len(row) > 0:
                        entry = extract_data_from_table(row)  # 테이블 행을 데이터로 변환
                        if any(entry.values()):
                            data.append(entry)
            # JSON 파일로 저장
            output_file = os.path.join(output_directory, f"{os.path.basename(pdf_path).replace('.pdf', '')}_page_{page_number + 1}_{safe_title_text}.json")
            with open(output_file, "w", encoding="utf-8") as json_file:
                json.dump(data, json_file, ensure_ascii=False, indent=4)
            # print(f"Parsed data for {pdf_path} page {page_number + 1}:", data)

def clean_text(text):  # 텍스트의 양끝 공백을 제거하는 함수
    return text.strip()

def format_sentence(sentence):  # 예문 형식을 정리하는 함수
    if re.search(r'\s햄', sentence):
        sentence = sentence.replace(".  햄", "\n햄")
    return sentence

def extract_words_from_pdf_by_page(pdf_path, output_dir="output_pages"):  # beyond.pdf용 : PyMuPDF를 사용하여 페이지별로 PDF 단어를 추출하는 함수
    document = fitz.open(pdf_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = format_sentence(page.get_text().replace('"', ''))
        text = text.replace(". 나", "\n나").replace("\n는", "는").replace(". 우", "\n우").replace("\n만", "만").replace("\n다", "다").replace("\n있다", "있다").replace("\n은편에", "은편에").replace("그녀는", "\n그녀는").replace("학습", "\n학습").replace("\nstudents", "students")
        
        # Lesson 제목을 뽑아서 파일 이름에 포함
        lesson_title = re.search(r'Lesson \d+\. (.+?)\s*\n', text)
        title_text = lesson_title.group(1) if lesson_title else f"Page_{page_num + 1}"
        safe_title_text = safe_filename(title_text)

        # 단어 정보 추출 패턴 정의
        pattern = r"(?P<단어>\w+)\s+(?P<품사>[a-z.]+)\s+(?P<단어_뜻>.+?)\n(?P<예문>.+?)\n(?P<예문_뜻>.+?)\n"
        if page_num == 12 or page_num == 23:
            pattern = r"(?P<단어>\w+(?:\s\w+)*)\s+(?P<품사>[a-z.]+)\s+(?P<단어_뜻>.+?)\n(?P<예문>.+?)\n(?P<예문_뜻>.+?)\n"
        matches = re.finditer(pattern, text, re.MULTILINE)
        
        page_words = []
        for match in matches:
            word_info = {
                "단어": clean_text(match.group("단어")),
                "단어 뜻": clean_text(match.group("단어_뜻")),
                "품사": clean_text(match.group("품사")),
                "예문": clean_text(match.group("예문")),
                "예문 뜻": clean_text(match.group("예문_뜻"))
            }
            if word_info["단어"] and word_info["예문"]:
                page_words.append(word_info)
        
        # JSON 파일 저장
        output_file = os.path.join(output_dir, f"{os.path.basename(pdf_path).replace('.pdf', '')}_page_{page_num + 1}_{safe_title_text}.json")
        with open(output_file, "w", encoding="utf-8") as json_file:
            json.dump(page_words, json_file, ensure_ascii=False, indent=4)
    
def process_pdfs(pdf_directory, output_directory, include_beyond=False, ):
    os.makedirs(output_directory, exist_ok=True)
    for pdf_file in tqdm(os.listdir(pdf_directory)):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        if pdf_file.endswith(".pdf"):
            if pdf_file == "T9EE70U04.pdf" and include_beyond: # beyond 전용
                print(f"{pdf_file}를 처리하는 중...")
                extract_words_from_pdf_by_page(pdf_path, output_dir=output_directory)
            else:
                pdf_to_json(pdf_path, output_directory)

# PDF 파일 처리
process_pdfs(PDF_DIR, OUT_DIR)


100%|██████████| 11/11 [00:54<00:00,  4.99s/it]


# 단어 / 예문 JSON 추출

In [1]:
import json
import os

# JSON 파일들이 있는 디렉토리를 정의합니다
JSON_DIR = "./json_words/data"  # JSON 파일들이 있는 폴더 경로로 변경하세요
OUT_WORD_JSON_FILE = "./json_words/combined/combined_words.json"
OUT_EXAMPLE_JSON_FILE = "./json_words/combined/combined_examples.json"

if not os.path.exists("./json_words/combined"):
    os.makedirs("./json_words/combined")

# 단어와 예문을 저장할 리스트
words_list = []
examples_list = []

# 디렉토리 내 모든 JSON 파일을 순회합니다
for filename in os.listdir(JSON_DIR):
    if filename.endswith(".json"):
        file_path = os.path.join(JSON_DIR, filename)
        
        # JSON 파일을 열고 로드합니다
        with open(file_path, "r", encoding="utf-8") as json_file:
            data = json.load(json_file)
            
            # 각 항목에서 "단어"와 "예문" 필드를 추출합니다
            for entry in data:
                word = entry.get("단어", "")
                example = entry.get("예문", "")
                
                # 단어 또는 예문이 있는 경우 각각의 리스트에 추가합니다
                if word:
                    words_list.append(word)
                if example and all(not char.isdigit() and char not in '가-힣' for char in example):  # 예문에 한글이 없는 경우만 추가
                    examples_list.append(example)

# 단어들을 새로운 JSON 파일로 저장합니다
with open(OUT_WORD_JSON_FILE, "w", encoding="utf-8") as output_words_json:
    json.dump(words_list, output_words_json, ensure_ascii=False, indent=4)

# 예문들을 새로운 JSON 파일로 저장합니다
with open(OUT_EXAMPLE_JSON_FILE, "w", encoding="utf-8") as output_examples_json:
    json.dump(examples_list, output_examples_json, ensure_ascii=False, indent=4)

print(f"단어들이 {OUT_WORD_JSON_FILE} 파일에 저장되었습니다.")
print(f"예문들이 {OUT_EXAMPLE_JSON_FILE} 파일에 저장되었습니다.")


단어들이 ./json_words/combined/combined_words.json 파일에 저장되었습니다.
예문들이 ./json_words/combined/combined_examples.json 파일에 저장되었습니다.
